In [ ]:
import os
import json
import numpy as np
from skimage import io as skio
from skimage.transform import resize
import matplotlib.pyplot as plt
from skimage.measure import label

In [ ]:
import javabridge as jv, bioformats as bf
jv.start_vm(class_path=bf.JARS, max_heap_size='4G')

def get_stack_from_series_id(czi_path, channel, series_id, dim):

    # Extraction a series from multi-position CZI

    img_raw = []
    for slice in range(dim[0]):
        img_raw.append(bf.load_image(path=czi_path, series=series_id, c=channel, z=slice, t=0, rescale=False))
    img_raw = np.array(img_raw).reshape(*dim)

    return img_raw

from skimage.filters import gaussian
from skimage.morphology import extrema, binary_dilation

def norm_and_smooth(img_original):

    # Uses Jianxu's normalization and Gaussian smooth to
    # preprocess the input image. Parameters have default
    # values used in the segmentation toolkit.

    smooth_sigma = 1.0
    scaling_params = [0.5, 18]

    img_norm = img_original.copy()
    mean = img_norm.mean()
    stdv = img_norm.std()
    strech_min = np.max([mean-scaling_params[0]*stdv, img_norm.min()])
    strech_max = np.min([mean+scaling_params[1]*stdv, img_norm.max()])
    img_norm[img_norm > strech_max] = strech_max
    img_norm[img_norm < strech_min] = strech_min
    img_norm = (img_norm-strech_min + 1e-8)/(strech_max-strech_min + 1e-8)
    img_norm = gaussian(image=img_norm, sigma=smooth_sigma)

    # img_norm may contain negative values

    return img_norm

In [ ]:
with open("../config-mix.json", "r") as fj:
    config_json = json.load(fj)
config_json_full = config_json["data"]

In [ ]:
for config_czi in config_json_full[0:10]:
    config_czi["raw_path"]

    #
    #
    #

    raw_path = os.path.join(config_czi["raw_path"],config_czi["raw_name"])
    position_id = np.random.randint(len(config_czi["position"]))
    seg_name = os.path.join(config_czi["seg_path"],config_czi["position"][position_id]["name"])
    if config_czi["modality"] == "880":
        series_id = 0
    else:
        series_id = int(seg_name.split("Scene")[1].split("-")[1]) - 1
    print("series_id:", series_id)

    #
    #
    #

    scale_factor = config_czi["xy_upsample_seg"]
    
    img_seg_all = skio.imread(seg_name)
    img_seg_all[ 0,:,:] = 0
    img_seg_all[-1,:,:] = 0
    nz, ny, nx = img_seg_all.shape
    img_raw = get_stack_from_series_id(czi_path=raw_path, channel=0, series_id=series_id, dim=(nz,scale_factor*ny,scale_factor*nx))
    img_raw_display = img_raw.max(axis=0)
    img_raw_display[0,0] = 2000
    plt.imshow(img_raw_display)
    plt.show()

    #
    #
    #

    if scale_factor > 1:
        img_seg_all_dtype = img_seg_all.dtype
        img_seg_all = resize(image=img_seg_all, output_shape=(nz,scale_factor*ny,scale_factor*nx), order=0, preserve_range=True, anti_aliasing=False, mode="constant")
        img_seg_all = img_seg_all.astype(img_seg_all_dtype)
    plt.imshow(img_seg_all.mean(axis=0))
    plt.show()

    #
    #
    #

    cell_id = np.random.choice(config_czi["position"][position_id]["cell_id"], size=1)
    print(cell_id)
    img_seg = img_seg_all.copy()
    img_seg[img_seg!=cell_id] = 0
    img_seg[img_seg==cell_id] = 1
    plt.imshow(img_seg.max(axis=0))
    plt.show()

    #
    #
    #

    print(img_seg.shape[0]*img_seg.shape[1]*img_seg.shape[2])
    img_seg = label(img_seg)
    print("#Comps:",np.unique(img_seg))
    print("Sizes:",np.bincount(img_seg.flat,minlength=img_seg.max()))
    largest_cc = 1 + np.argmax(np.bincount(img_seg.flat)[1:])
    print("Largest CC:", largest_cc)

    #
    #
    #

    if img_seg.max() > 1:
        img_seg[img_seg!=largest_cc] = 0
        img_seg[img_seg>0] = 1
    plt.imshow(img_seg.max(axis=0))
    plt.imshow(img_seg.max(axis=1))
    plt.show()
    
    #
    #
    #

    img_input = img_raw * img_seg
    pxl_z, pxl_y, pxl_x = np.nonzero(img_input)
    img_raw_crop = img_input[pxl_z.min():(pxl_z.max()+1),pxl_y.min():(pxl_y.max()+1),pxl_x.min():(pxl_x.max()+1)]
    plt.imshow(img_raw_crop.max(axis=1))
    plt.show()
    
    #
    #
    #

    nz_crop, ny_crop, nx_crop = img_raw_crop.shape
    nz_crop_new = np.int((config_czi["pixel_size_z"]/config_czi["pixel_size_xy"])*nz_crop)
    print(img_raw_crop.shape, img_raw_crop.dtype)
    img_raw_crop_resize = resize(image=img_raw_crop, output_shape=(nz_crop_new,ny_crop,nx_crop), preserve_range=True, anti_aliasing=True, mode="constant")
    img_raw_crop_resize = img_raw_crop_resize.astype(np.int64)
    print(img_raw_crop_resize.shape)
    print(img_raw_crop_resize.dtype,img_raw_crop_resize.min(),img_raw_crop_resize.max())
    fig, ax = plt.subplots(nrows=1,ncols=2)
    ax[0].imshow(img_raw_crop_resize.max(axis=0))
    ax[1].imshow(img_raw_crop_resize.max(axis=1))
    plt.show()
    
    img_input = img_raw_crop_resize

    #
    #
    #

    img_norm = norm_and_smooth(img_original=img_input)

    img_max = extrema.h_maxima(img_norm, h=0.25)
    z_pxl, y_pxl, x_pxl = np.nonzero(img_max)
    print(len(z_pxl))

    #img_input[0,0] = 2000
    avg = img_input[img_input>0].mean()
    std = img_input[img_input>0].std()
    fig, ax = plt.subplots(nrows=2,ncols=2)
    fig.suptitle(config_czi["cell_type"]+"-"+config_czi["structure"]+"-"+config_czi["modality"]+"-"+config_czi["condition"])
    ax[0,0].imshow(img_input.max(axis=0))
    ax[0,0].plot(x_pxl,y_pxl,"r+")
    ax[0,1].imshow(img_input.max(axis=0)>(avg+2.0*std))
    ax[1,0].imshow(img_input.max(axis=1))
    ax[1,0].plot(x_pxl,z_pxl,"r+")
    ax[1,1].imshow(img_input.max(axis=1)>(avg+2.0*std))
    plt.show()

In [ ]:
config_czi